In [33]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from matplotlib import pyplot as plt

In [2]:
daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34,31]

tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [3]:
for label in tf_dataset:
    print(label)

tf.Tensor(21, shape=(), dtype=int32)
tf.Tensor(22, shape=(), dtype=int32)
tf.Tensor(-108, shape=(), dtype=int32)
tf.Tensor(31, shape=(), dtype=int32)
tf.Tensor(-1, shape=(), dtype=int32)
tf.Tensor(32, shape=(), dtype=int32)
tf.Tensor(34, shape=(), dtype=int32)
tf.Tensor(31, shape=(), dtype=int32)


In [4]:
for label in tf_dataset.as_numpy_iterator():
    print(label)

21
22
-108
31
-1
32
34
31


In [5]:
tf_dataset = tf_dataset.filter(lambda x: x>0)

In [6]:
for label in tf_dataset.as_numpy_iterator():
    print(label)

21
22
31
32
34
31


In [7]:
tf_dataset = tf_dataset.map(lambda x: x*72)

In [8]:
for label in tf_dataset.as_numpy_iterator():
    print(label)

1512
1584
2232
2304
2448
2232


In [9]:
tf_dataset = tf_dataset.shuffle(3)

In [10]:
for label in tf_dataset.as_numpy_iterator():
    print(label)

1584
2232
1512
2304
2232
2448


In [11]:
tf_dataset = tf_dataset.batch(2)

In [12]:
for label in tf_dataset.as_numpy_iterator():
    print(label)

[2232 1512]
[1584 2304]
[2448 2232]


In [13]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

tf_dataset = tf_dataset.filter(lambda x: x>0).map(lambda y: y*72).shuffle(3).batch(2)

In [14]:
for label in tf_dataset.as_numpy_iterator():
    print(label)

[1512 2304]
[1584 2232]
[2232 2448]


In [15]:
image_ds = tf.data.Dataset.list_files('images/*/*',shuffle=False)


In [16]:
image_count = len(image_ds)
image_count

130

In [17]:
type(image_ds)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [19]:
for label in image_ds.take(3):
    print(label.numpy())

b'images\\cat\\20 Reasons Why Cats Make the Best Pets....jpg'
b'images\\cat\\7 Foods Your Cat Can_t Eat.jpg'
b'images\\cat\\A cat appears to have caught the....jpg'


In [20]:
image_ds = image_ds.shuffle(200)
for label in image_ds.take(3):
    print(label.numpy())

b'images\\cat\\What to do if your cat is marking....jpg'
b'images\\dog\\25 Low-Maintenance Dog Breeds for....jpg'
b'images\\dog\\The Black Dog Tavern Company _ Life off....jpg'


In [21]:
class_names = ["cat","dog"]

In [24]:
train_size = int(image_count*0.8)
train_ds = image_ds.take(train_size)
test_ds = image_ds.skip(train_size)

In [25]:
len(train_ds), len(test_ds)

(104, 26)

In [26]:
def get_label(file_path):
    
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2]
    

In [28]:
get_label('images\\cat\\What to do if your cat is marking....jpg')

<tf.Tensor: shape=(), dtype=string, numpy=b'cat'>

In [30]:
def image_processing(file_path):
    
    label = get_label(file_path)
    
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128,128])
    
    return img, label

In [35]:
img, label = image_processing('images\\cat\\What to do if your cat is marking....jpg')
img , label

(<tf.Tensor: shape=(128, 128, 3), dtype=float32, numpy=
 array([[[197.22656, 189.22656, 187.22656],
         [187.22656, 177.22656, 175.22656],
         [195.9336 , 186.9336 , 181.9336 ],
         ...,
         [181.58594, 166.58594, 162.58594],
         [182.16016, 171.16016, 167.16016],
         [177.72656, 166.72656, 164.72656]],
 
        [[192.17969, 184.17969, 181.17969],
         [192.14062, 182.14062, 180.14062],
         [195.83984, 186.51953, 182.48047],
         ...,
         [181.23828, 166.23828, 162.5586 ],
         [179.66016, 168.66016, 164.66016],
         [175.85938, 163.89844, 162.21875]],
 
        [[191.9336 , 184.80078, 179.19922],
         [194.4336 , 186.16797, 183.30078],
         [200.60156, 190.60156, 189.03516],
         ...,
         [177.23438, 162.23438, 159.23438],
         [185.10156, 174.10156, 170.10156],
         [170.9336 , 156.9336 , 154.19922]],
 
        ...,
 
        [[198.46875, 187.60156, 185.60156],
         [192.69922, 182.5664 , 181.4336 ]

In [37]:
train_ds = train_ds.map(image_processing)
test_ds = test_ds.map(image_processing)

In [38]:
for img, label in train_ds.take(3):
    print(img)
    print(label)

tf.Tensor(
[[[ 80.54791    84.75104    25.954163 ]
  [ 83.27655    74.63513     4.559387 ]
  [ 82.961      71.91797     8.093811 ]
  ...
  [ 58.083313   46.083313   24.083313 ]
  [ 55.78125    44.78125    25.398438 ]
  [ 36.121643   30.138306   15.551331 ]]

 [[ 65.34766    67.71094     5.2460938]
  [ 96.67969    88.529724    6.6516724]
  [ 98.36725    85.066345    8.257874 ]
  ...
  [ 71.6991     52.222534   27.484253 ]
  [ 55.882812   42.492188   27.664062 ]
  [ 65.015625   62.5578     61.619568 ]]

 [[ 66.953186   66.359436    1.2435303]
  [108.36322    99.29303     3.6210327]
  [115.375     106.94922     9.533752 ]
  ...
  [ 91.44531    70.06641    26.760315 ]
  [ 50.390625   37.98047    17.210938 ]
  [111.9762    113.012695  113.03998  ]]

 ...

 [[179.79291   167.01166   104.42572  ]
  [184.01929   164.58179   111.19116  ]
  [130.38452   135.17609    66.44061  ]
  ...
  [166.57672   160.59235   109.706726 ]
  [154.08612   144.50793    97.5979   ]
  [161.60162   167.02094    81.55

In [39]:
def scale(img,label):
    return img/255, label

In [40]:
train_ds = train_ds.map(scale)

In [42]:
for img, label in train_ds.take(3):
    print(label)
    print(img)

tf.Tensor(b'cat', shape=(), dtype=string)
tf.Tensor(
[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 ...

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]], shape=(128, 128, 3), dtype=float32)
tf.Tensor(b'dog', shape=(), dtype=string)
tf.Tensor(
[[[0.03999694 0.07136948 0.01254596 1.        ]
  [0.05098039 0.07450981 0.01960784 1.        ]
  [0.05518835 0.07259498 0.01589008 1.        ]
  ...
  [0.19096535 0.24225213 0.11636364 1.        ]
  [0.16562141 0.2051187  0.10822838 1.        ]
  [0.15234135 0.17487973 0.08643236 1.        ]]

 [[0.0508308  0.07985959 0.01972453 1. 